# Day 1: The setup with kaggle mnist dataset

In [1]:
import kaggle
import keras
import torch
%load_ext tensorboard
# check for gpu
!nvidia-smi
# check for cuda
!nvcc --version
# use gpu
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.

Purpose of this notebook is to:
* run kaggle mnist locally
* on gpu
* and upload the results to kaggle.

##  Download the dataset from kaggle

In [ ]:
import os
import zipfile

# Define the directory where you want to download the data
data_dir = "MyDataset/mnist/raw"  # './' represents the current directory

# Check if the directory exists
if not os.path.exists(data_dir):
    # If not, create the directory
    os.makedirs(data_dir)

# Move to that directory
os.chdir(data_dir)
competition_name = "digit-recognizer"
# Download the data
os.system("kaggle competitions download -c " + competition_name)

# Unzip the data
with zipfile.ZipFile("digit-recognizer.zip","r") as zip_ref:
    zip_ref.extractall(".")

os.chdir("../../..")

## Load Data And prepare the data

In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
train_df = pd.read_csv('MyDataset/mnist/raw/train.csv')
test_df = pd.read_csv('MyDataset/mnist/raw/test.csv')

# Split features and labels
y_train = train_df["label"]
x_train = train_df.drop(labels = ["label"], axis = 1)

# Convert to numpy arrays
x_train = x_train.values
y_train = y_train.values

# Test data
x_test = test_df.values



### Inspet the data

In [ ]:
train_df.head()

### Prepare the data

In [ ]:
# Make Validation set
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state=2)

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)
x_val = x_val.reshape(-1, 28, 28, 1)
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

## Make model

In [ ]:
num_classes = 10
input_shape = (28, 28, 1)

model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)
model.summary()

### Compile the model

In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="acc"),
    ],
)

### Callbacks   

In [ ]:
callbacks = [ 
    keras.callbacks.EarlyStopping(monitor="val_loss",patience=10),
    keras.callbacks.ModelCheckpoint("mnist_model.keras", save_best_only=True),
    keras.callbacks.TensorBoard(log_dir="./logs"),
]
%tensorboard --logdir=logs

### Fit the model

In [ ]:
batch_size = 128
epochs = 20

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(x_val, y_val),

)

score = model.evaluate(x_val, y_val, verbose=0)

In [ ]:

import pandas as pd
# Assuming 'predictions' is an array containing your model's predictions
predictions = model.predict(x_test)


In [ ]:
class_predictions = np.argmax(predictions, axis=1)
# Create a DataFrame with the prediction results
# 'ImageId' is a common column name in MNIST-like competitions
submission = pd.DataFrame({
    "ImageId": list(range(1, len(class_predictions) + 1)),
    "Label": class_predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('my_submission.csv', index=False)

In [ ]:
#!kaggle competitions submit -c digit-recognizer -f my_submission.csv -m "First submission"


In [ ]:
!kaggle competitions submissions digit-recognizer